# Breast Cancer Detection


*The following is a solution to the RSNA Screeening Mammography Breast Cancer Detection Dataset*

In [1]:
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_datasets as tfds
import keras
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split

import cv2

import os
import logging
import random
import pickle
import math

from multiprocessing import cpu_count


E0000 00:00:1728237428.250045      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1006 17:57:08.258215161      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1006 17:57:08.258229003      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1006 17:57:08.258232370      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1006 17:57:08.258234770      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1006 17:57:08.258237036      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on CPU or GPU')
    TPU = None

if TPU:
    IS_TPU = True
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    STRATEGY = tf.distribute.experimental.TPUStrategy(TPU)
else:
    IS_TPU = False
    STRATEGY = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

#clear_output()
N_REPLICAS = STRATEGY.num_replicas_in_sync
print(f'N_REPLICAS: {N_REPLICAS}, IS_TPU: {IS_TPU}')

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1728237445.742770      13 service.cc:145] XLA service 0x5b59ed501900 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728237445.742816      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1728237445.742821      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1728237445.742823      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1728237445.742826      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1728237445.742829      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1728237445.742832      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1728237445.742834      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1728237445.742837      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


N_REPLICAS: 8, IS_TPU: True


In [3]:
# Image dimensions
IMG_HEIGHT = 1456
IMG_WIDTH = 728
N_CHANNELS = 1
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 1)
N_SAMPLES_TFRECORDS = 548

# Peak Learning Rate
EPOCHS = 15

# Batch size
BATCH_SIZE = 8 * N_REPLICAS

# Augmentation
BRIGHTNESS = 0.10
CONTRAST = (0.90, 1.10)
JPEG_QUALITY = (90, 100)
CROP_RATIO = (0.80, 1.00)

IS_INTERACTIVE = os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
VERBOSE = 1 if IS_INTERACTIVE else 2

# Tensorflow AUTO flag
AUTO = tf.data.experimental.AUTOTUNE

print(f'BATCH_SIZE: {BATCH_SIZE}')

BATCH_SIZE: 64


In [4]:
'''IMG_HEIGHT = 512
IMG_WIDTH = 256'''

'IMG_HEIGHT = 512\nIMG_WIDTH = 256'

In [5]:
MIXED_PRECISION = False
DEVICE = 'TPU'

if MIXED_PRECISION:
    if 'TPU' in DEVICE:
        policy_type = 'mixed_bfloat16'
    else:
        policy_type = 'mixed_float16'
else:
    policy_type = 'float32'
policy = tf.keras.mixed_precision.Policy(policy_type)
tf.keras.mixed_precision.set_global_policy(policy)
print(f'Computation dtype: {tf.keras.mixed_precision.global_policy().compute_dtype}')
print(f'Variable dtype: {tf.keras.mixed_precision.global_policy().variable_dtype}')

Computation dtype: float32
Variable dtype: float32


# Preporcessing

In [6]:
# TFRecord file paths
tfrecords = sorted(tf.io.gfile.glob('/kaggle/input/dataset-rsna-bcd-1456x728-final-tfrecords/*.tfrecords'))
print(f'Found {len(tfrecords)} TFRecords')

# Train Test Split
train_tfrecords, valid_tfrecords = train_test_split(
    tfrecords,
    train_size=0.80,
    random_state=42,
    shuffle=True)
print(f'# Train TFRecords: {len(train_tfrecords)}, # Valid TFRecords: {len(valid_tfrecords)}')

Found 100 TFRecords
# Train TFRecords: 80, # Valid TFRecords: 20


In [7]:
def create_dataset(tfrecords, batch_size=32, valid=False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    # Create dataset path/label
    dataset = tf.data.TFRecordDataset(tfrecords,num_parallel_reads=AUTO,compression_type='GZIP')

    # Decode mapping
    dataset = dataset.map(decoder, num_parallel_calls=AUTO)
    # Val/Debug cases
    if not valid:
        dataset = dataset.filter(undersample_majority)
        
        dataset = dataset.map(augmenter, num_parallel_calls=AUTO)
        dataset = dataset.with_options(ignore_order)
        
        dataset = dataset.shuffle(1024)
        dataset = dataset.repeat()
    if valid:
        dataset = dataset.map(expand_label, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def decoder(record_bytes):
    features = tf.io.parse_single_example(record_bytes, {
        'image': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.int64),
        'patient_id': tf.io.FixedLenFeature([], tf.int64)})
    img = tf.io.decode_png(features['image'], channels=N_CHANNELS)
    img = tf.reshape(img, [IMG_HEIGHT, IMG_WIDTH, N_CHANNELS])
    label = features['target']
    return img, label

def undersample_majority(img, label):
    return label == 1 or tf.random.uniform([]) > 2/3

def tf_rand_int(minval, maxval, dtype=tf.int64):
    minval = tf.cast(minval, dtype)
    maxval = tf.cast(maxval, dtype)
    return tf.random.uniform(shape=(), minval=minval, maxval=maxval, dtype=dtype)

def augmenter(img, label):
    # Pixels
    img = tf.image.random_brightness(img, BRIGHTNESS)
    img = tf.image.random_contrast(img, *CONTRAST)
    img = tf.image.random_jpeg_quality(img, *JPEG_QUALITY)
    # Crop
    ratio = tf.random.uniform([], *CROP_RATIO)
    img_height_crop = tf.cast(ratio * IMG_HEIGHT, tf.int32)
    img_width_crop = tf.cast(ratio * IMG_WIDTH, tf.int32)
    img_height_offset = tf_rand_int(0, IMG_HEIGHT - img_height_crop)
    img_width_offset = 0
    img = tf.slice(img, [img_height_offset, img_width_offset, 0],
                   [img_height_crop, img_width_crop, N_CHANNELS])
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    # Clip
    img = tf.clip_by_value(img, 0, 255)
    img = tf.cast(img, tf.uint8)
    img = img / 255
    label = tf.expand_dims(label, axis=-1)
    return img, label

def expand_label(img,label):
    img = img / 255
    label = tf.expand_dims(label, axis=-1)
    return img, label


In [8]:
# Get Train/Validation datasets
train_dataset = create_dataset(train_tfrecords, valid=False)
valid_dataset = create_dataset(valid_tfrecords, valid=True)

TRAIN_STEPS_PER_EPOCH = (len(train_tfrecords) * N_SAMPLES_TFRECORDS) // BATCH_SIZE
VAL_STEPS_PER_EPOCH = (len(valid_tfrecords) * N_SAMPLES_TFRECORDS) // BATCH_SIZE
print(f'TRAIN_STEPS_PER_EPOCH: {TRAIN_STEPS_PER_EPOCH}, VAL_STEPS_PER_EPOCH: {VAL_STEPS_PER_EPOCH}')

TRAIN_STEPS_PER_EPOCH: 685, VAL_STEPS_PER_EPOCH: 171


In [9]:
# Sanity checking
def check_dataset(dataset):
    image, label = next(iter(dataset))
    image = image.numpy()
    #clear_output()
    print(f"X_batch shape: {image.shape}, y_batch shape: {label.shape}")
    print(f"X_batch dtype: {image.dtype}, y_batch dtype: {label.dtype}")
    print(f"X_batch min: {image.min():.2f}, max: {image.max():.2f}")

check_dataset(train_dataset)

X_batch shape: (64, 1456, 728, 1), y_batch shape: (64, 1)
X_batch dtype: float32, y_batch dtype: <dtype: 'int64'>
X_batch min: 0.00, max: 1.00


# Creating the Model

In [11]:
@keras.saving.register_keras_serializable()
class pFBeta(tf.keras.Metric):
    def __init__(self, beta=1, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = 1e-10
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.keras.ops.cast(y_true, tf.float32)
        y_pred = tf.keras.ops.clip(y_pred, 0, 1)
        pos = tf.keras.ops.cast(tf.keras.ops.sum(y_true), tf.float32)
        ctp = tf.keras.ops.cast(tf.keras.ops.sum(y_pred[y_true == 1]), tf.float32)
        cfp = tf.keras.ops.cast(tf.keras.ops.sum(y_pred[y_true == 0]), tf.float32)
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta2 = self.beta * self.beta
        prec = self.ctp / (self.ctp + self.cfp + self.epsilon)
        reca = self.ctp / (self.pos + self.epsilon)
        return (1 + beta2) * prec * reca / ((beta2 * prec + reca) + self.epsilon) 
    
    def reset_state(self):
        self.pos.assign(0.)
        self.ctp.assign(0.)
        self.cfp.assign(0.)

In [23]:
def model():
        
    # Architecture
    inputs = keras.Input((1456, 728, 1), name='inputs')

    x = keras.ops.cast(inputs, tf.float32)
    x = keras.ops.repeat(x, repeats=3, axis=3)

    efficientnet = keras.applications.EfficientNetB4(
        weights='imagenet',
        input_shape=(1456, 728, 3),
        include_top=False,
        pooling="avg",
    )(x)


    x = keras.layers.Dropout(0.30)(efficientnet)
    outputs_1 = keras.layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs_1, name="model")
    #model.load_weights("whatthesssigma.weights.h5")

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
                      loss=tf.keras.losses.BinaryCrossentropy(), 
                      metrics=[pFBeta(),
                       tf.metrics.AUC(name='AUC'),
                       tf.metrics.F1Score(threshold=0.50, name='F1'),
                       tf.metrics.Precision(name='Prec'),
                       tf.metrics.Recall(name='Reca'),
                       tf.metrics.BinaryAccuracy(name='BinAcc')])
    

    # Print the model summary
    print(model.summary())
    return model

    
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit("autoclustering")

model = model()

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "small_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inputs_1 (InputLayer)           │ (None, 1456, 728, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_1 (Cast)                   │ (None, 1456, 728, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat (Repeat)                 │ (None, 1456, 728, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 1792)           │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,793 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,675,616 (67.43 MB)

 Trainable params: 17,550,409 (66.95 MB)

 Non-trainable params: 125,207 (489.09 KB)

None


In [ ]:
history = model.fit(
    x=train_dataset,
    epochs=25,
    validation_data=valid_dataset,
    class_weight={0: 1, 1: 5},
    steps_per_epoch=TRAIN_STEPS_PER_EPOCH)
with STRATEGY.scope():
    model.save("/kaggle/working/model_v8.keras")